In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import ipywidgets as widgets
from IPython.display import display

sns.set_style("whitegrid")
%matplotlib inline

try:
    df_1 = pd.read_csv('data/SB11-2016-1.txt', sep='|', encoding='latin1', dtype={'COLE_COD_DANE': str})
    df_2 = pd.read_csv('data/SB11-2016-2.txt', sep='|', encoding='latin1', dtype={'COLE_COD_DANE': str})
    

    df = pd.concat([df_1, df_2], ignore_index=True)
    print("Datos cargados y unidos correctamente.")
except FileNotFoundError:
    print("Error: Asegúrate de que los archivos 'SB11-2016-1.txt' y 'SB11-2016-2.txt' están en la carpeta 'data'.")
    df = pd.DataFrame() # Crear un DataFrame vacío para evitar errores

if not df.empty:
    print(f"\nDimensiones de los datos: {df.shape[0]} filas, {df.shape[1]} columnas.")


    years = df['ANO_PRESENTACION'].unique()
    periods = df['PERIODO'].unique()
    print(f"Años de presentación: {years}")
    print(f"Períodos de evaluación: {periods}")

    
    variables_interes = [
        'ESTU_GENERO', 'ESTU_EDAD', 'FAMI_ESTRATOVIVIENDA', 'FAMI_NIVEL_SISBEN',
        'FAMI_EDUCA_PADRE', 'FAMI_EDUCA_MADRE', 'FAMI_OCUPA_PADRE', 'FAMI_OCUPA_MADRE',
        'COLE_NATURALEZA', 'COLE_CARACTER', 'COLE_BILINGUE', 'PUNT_GLOBAL',
        'PUNT_LECTURA_CRITICA', 'PUNT_MATEMATICAS', 'PUNT_C_NATURALES', 'PUNT_SOCIALES_CIUDADANAS',
        'PUNT_INGLES'
    ]
    
    
    df_analisis = df[variables_interes].copy()
    print("\nVariables de interés seleccionadas:")
    print(df_analisis.columns.tolist())

    
    missing_data = df_analisis.isnull().sum() / len(df_analisis) * 100
    print("\nPorcentaje de datos faltantes por columna:")
    print(missing_data[missing_data > 0].sort_values(ascending=False))
    

    print("\nPropuesta para manejo de datos faltantes:")
    print("- Para variables numéricas como PUNT_GLOBAL, se pueden imputar con la mediana.")
    print("- Para variables categóricas, si el porcentaje es bajo, se pueden eliminar las filas. Si es alto, imputar con la moda.")
    print("- La estrategia dependerá del porcentaje y la naturaleza de cada variable.")

   if not df_analisis.empty:
    print("\nEstadísticas descriptivas de los puntajes globales:")
    print(df_analisis['PUNT_GLOBAL'].describe())

    plt.figure(figsize=(10, 6))
    sns.histplot(df_analisis['PUNT_GLOBAL'].dropna(), kde=True, bins=50)
    plt.title('Distribución del Puntaje Global en las Pruebas Saber 11', fontsize=16)
    plt.xlabel('Puntaje Global', fontsize=12)
    plt.ylabel('Frecuencia', fontsize=12)
    plt.show()


    plt.figure(figsize=(12, 8))
    sns.violinplot(x='FAMI_ESTRATOVIVIENDA', y='PUNT_GLOBAL', data=df_analisis.dropna(subset=['FAMI_ESTRATOVIVIENDA', 'PUNT_GLOBAL']))
    plt.title('Impacto del Estrato de Vivienda en el Puntaje Global', fontsize=16)
    plt.xlabel('Estrato de Vivienda', fontsize=12)
    plt.ylabel('Puntaje Global', fontsize=12)
    plt.show()
    
    estrato_puntaje_promedio = df_analisis.groupby('FAMI_ESTRATOVIVIENDA')['PUNT_GLOBAL'].mean().sort_values()
    print("\nPuntaje promedio por Estrato de Vivienda:")
    print(estrato_puntaje_promedio)


    print("\nConclusión de la Fase 2:")
    print("Las visualizaciones y estadísticas descriptivas sugieren una clara relación entre el estrato socioeconómico de los estudiantes y su desempeño en la prueba. Los puntajes promedio tienden a ser más altos en estratos más altos. Esto indica que la desigualdad socioeconómica podría ser un factor clave en los resultados académicos. Por lo tanto, un análisis más profundo se centraría en el impacto de variables como el estrato de vivienda, el nivel educativo de los padres y el tipo de colegio (naturaleza, carácter).")

    if not df_analisis.empty:
    
    df_modelo = df_analisis.dropna(subset=['PUNT_GLOBAL', 'FAMI_ESTRATOVIVIENDA', 'FAMI_EDUCA_PADRE', 'COLE_NATURALEZA'])
    
    
    df_modelo = pd.get_dummies(df_modelo, columns=['FAMI_ESTRATOVIVIENDA', 'FAMI_EDUCA_PADRE', 'COLE_NATURALEZA'], drop_first=True)
    
    
    X = df_modelo.drop('PUNT_GLOBAL', axis=1)
    y = df_modelo['PUNT_GLOBAL']
    
    
    X_numericas = X.select_dtypes(include=np.number)

    
    X_train, X_test, y_train, y_test = train_test_split(X_numericas, y, test_size=0.3, random_state=42)
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    
  
    y_pred = model.predict(X_test)
    
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    print("\nDesempeño del Modelo de Regresión Lineal:")
    print(f"Error Cuadrático Medio (MSE): {mse:.2f}")
    print(f"Coeficiente de Determinación (R²): {r2:.2f}")
    
    
    coefs = pd.DataFrame(model.coef_, X_numericas.columns, columns=['Coeficiente'])
    print("\nCoeficientes del modelo (importancia de las variables):")
    print(coefs.sort_values(by='Coeficiente', ascending=False))
    
    
    print("\nConclusión de la Fase 3:")
    print(f"El modelo de regresión lineal muestra un R² de {r2:.2f}, lo que indica que aproximadamente el {r2*100:.2f}% de la varianza en los puntajes globales es explicada por las variables sociodemográficas seleccionadas. Las variables categóricas, como el estrato de vivienda y la educación de los padres, resultan ser las más influyentes, lo que valida la hipótesis inicial de que los factores socioeconómicos tienen un impacto significativo en el desempeño académico de los estudiantes. Este hallazgo es crucial para los stakeholders del ICFES, ya que resalta la necesidad de políticas públicas que aborden la desigualdad educativa.")

    
if not df_analisis.empty:
    # Crear un widget para interactuar con la variable 'PUNT_GLOBAL'
    estratos = sorted(df_analisis['FAMI_ESTRATOVIVIENDA'].dropna().unique())
    estratos = [e for e in estratos if e != 'Sin Estrato']
    
    estrato_widget = widgets.Dropdown(
        options=estratos,
        value=estratos[0],
        description='Estrato de Vivienda:',
        disabled=False,
    )
    
   
    def plot_puntajes_por_estrato(estrato):
        plt.figure(figsize=(10, 6))
        
        
        df_filtered = df_analisis[df_analisis['FAMI_ESTRATOVIVIENDA'] == estrato].dropna(subset=['PUNT_GLOBAL'])
        
        
        sns.histplot(df_filtered['PUNT_GLOBAL'], kde=True, bins=50)
        plt.title(f'Distribución de Puntajes Globales para el Estrato {estrato}', fontsize=16)
        plt.xlabel('Puntaje Global', fontsize=12)
        plt.ylabel('Frecuencia', fontsize=12)
        
        
        mean_score = df_filtered['PUNT_GLOBAL'].mean()
        plt.axvline(mean_score, color='r', linestyle='--', label=f'Media: {mean_score:.2f}')
        plt.legend()
        
        plt.show()
    
    
    print("\nHerramienta Interactiva: Distribución de Puntajes por Estrato Socioeconómico")
    print("Utiliza el menú desplegable para ver cómo cambia la distribución de los puntajes globales según el estrato de vivienda.")
    widgets.interactive(plot_puntajes_por_estrato, estrato=estrato_widget)

    

IndentationError: unindent does not match any outer indentation level (<string>, line 59)